# Hybrid Recommender System

This notebook combines collaborative filtering and content-based approaches for hybrid recommendations.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Set display options
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')

## Load Required Data and Models

In [ ]:
# Load user-item matrix
# user_item_matrix = pd.read_csv('../results/user_item_matrix.csv', index_col=0)

# Load similarity matrices from previous notebooks
# user_similarity = pd.read_csv('../results/user_similarity.csv', index_col=0)
# item_similarity_cf = pd.read_csv('../results/item_similarity.csv', index_col=0)
# item_similarity_content = pd.read_csv('../results/content_item_similarity.csv', index_col=0)

# print("Data loaded successfully")

## Weighted Hybrid Method

In [ ]:
def weighted_hybrid_recommendations(user_id, 
                                   collaborative_recs,
                                   content_recs,
                                   alpha=0.5,
                                   top_n=10):
    """
    Combine collaborative and content-based recommendations using weighted approach.
    
    Args:
        user_id: Target user ID
        collaborative_recs: DataFrame with collaborative filtering recommendations
        content_recs: DataFrame with content-based recommendations
        alpha: Weight for collaborative filtering (1-alpha for content-based)
        top_n: Number of recommendations
        
    Returns:
        DataFrame with hybrid recommendations
    """
    # Normalize scores to [0, 1]
    collab_scores = collaborative_recs.set_index('item_id')['score']
    content_scores = content_recs.set_index('item_id')['score']
    
    # Normalize
    collab_scores = (collab_scores - collab_scores.min()) / (collab_scores.max() - collab_scores.min())
    content_scores = (content_scores - content_scores.min()) / (content_scores.max() - content_scores.min())
    
    # Combine scores
    all_items = set(collab_scores.index) | set(content_scores.index)
    hybrid_scores = {}
    
    for item in all_items:
        collab_score = collab_scores.get(item, 0)
        content_score = content_scores.get(item, 0)
        hybrid_scores[item] = alpha * collab_score + (1 - alpha) * content_score
    
    # Sort and get top-N
    sorted_items = sorted(hybrid_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
    
    recommendations = pd.DataFrame(sorted_items, columns=['item_id', 'hybrid_score'])
    return recommendations

# Example usage
# Assuming you have collaborative_recs and content_recs from previous notebooks
# hybrid_recs = weighted_hybrid_recommendations(user_id, collaborative_recs, content_recs, alpha=0.6)
# print("Hybrid Recommendations:")
# print(hybrid_recs)

## Switching Hybrid Method

In [ ]:
def switching_hybrid_recommendations(user_id,
                                    user_item_matrix,
                                    collaborative_recs,
                                    content_recs,
                                    min_ratings=10,
                                    top_n=10):
    """
    Switch between collaborative and content-based based on user profile.
    
    Args:
        user_id: Target user ID
        user_item_matrix: User-item rating matrix
        collaborative_recs: Collaborative filtering recommendations
        content_recs: Content-based recommendations
        min_ratings: Minimum ratings to use collaborative filtering
        top_n: Number of recommendations
        
    Returns:
        DataFrame with recommendations and method used
    """
    # Check user's rating count
    user_ratings = user_item_matrix.loc[user_id]
    rating_count = (user_ratings > 0).sum()
    
    # Switch based on rating count
    if rating_count >= min_ratings:
        # Use collaborative filtering for active users
        recommendations = collaborative_recs.head(top_n).copy()
        recommendations['method'] = 'collaborative'
    else:
        # Use content-based for new/inactive users
        recommendations = content_recs.head(top_n).copy()
        recommendations['method'] = 'content-based'
    
    return recommendations

# Example usage
# switching_recs = switching_hybrid_recommendations(user_id, user_item_matrix, collaborative_recs, content_recs)
# print(f"\nSwitching Hybrid Recommendations (Method: {switching_recs['method'].iloc[0]}):")
# print(switching_recs)

## Feature Augmentation Hybrid

In [ ]:
def feature_augmentation_hybrid(user_id,
                               user_item_matrix,
                               item_similarity_cf,
                               item_similarity_content,
                               top_n=10):
    """
    Augment collaborative features with content features.
    
    Args:
        user_id: Target user ID
        user_item_matrix: User-item rating matrix
        item_similarity_cf: Item similarity from collaborative filtering
        item_similarity_content: Item similarity from content-based
        top_n: Number of recommendations
        
    Returns:
        DataFrame with recommendations
    """
    user_ratings = user_item_matrix.loc[user_id]
    rated_items = user_ratings[user_ratings > 0]
    unrated_items = user_ratings[user_ratings == 0].index
    
    predictions = {}
    
    for item in unrated_items:
        cf_score = 0
        content_score = 0
        
        # Calculate collaborative score
        for rated_item, rating in rated_items.items():
            cf_score += item_similarity_cf.loc[item, rated_item] * rating
            content_score += item_similarity_content.loc[item, rated_item] * rating
        
        # Combine scores (average)
        if len(rated_items) > 0:
            predictions[item] = (cf_score + content_score) / (2 * len(rated_items))
    
    # Get top-N recommendations
    sorted_items = sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:top_n]
    recommendations = pd.DataFrame(sorted_items, columns=['item_id', 'augmented_score'])
    
    return recommendations

# Example usage
# augmented_recs = feature_augmentation_hybrid(user_id, user_item_matrix, item_similarity_cf, item_similarity_content)
# print("\nFeature Augmentation Hybrid Recommendations:")
# print(augmented_recs)

## Evaluate Hybrid Approaches

In [ ]:
def evaluate_hybrid_methods(test_users, methods_dict, actual_ratings):
    """
    Evaluate different hybrid methods.
    
    Args:
        test_users: List of user IDs to test
        methods_dict: Dictionary of method names and their recommendation functions
        actual_ratings: Actual ratings for evaluation
        
    Returns:
        DataFrame with evaluation metrics
    """
    results = {}
    
    for method_name, method_func in methods_dict.items():
        predictions = []
        actuals = []
        
        for user_id in test_users:
            recs = method_func(user_id)
            # Compare with actual ratings
            # This is a simplified version
            predictions.extend(recs['predicted_rating'].values if 'predicted_rating' in recs.columns else [])
            actuals.extend(actual_ratings.loc[user_id, recs['item_id']].values)
        
        # Calculate metrics
        if len(predictions) > 0:
            rmse = np.sqrt(mean_squared_error(actuals, predictions))
            mae = mean_absolute_error(actuals, predictions)
            results[method_name] = {'RMSE': rmse, 'MAE': mae}
    
    return pd.DataFrame(results).T

## Visualize Hybrid Performance

In [ ]:
# Compare different alpha values for weighted hybrid
# alphas = [0.0, 0.2, 0.4, 0.5, 0.6, 0.8, 1.0]
# performance = []

# for alpha in alphas:
#     # Calculate performance for each alpha
#     # This is a placeholder - implement actual evaluation
#     pass

# plt.figure(figsize=(10, 6))
# plt.plot(alphas, performance, marker='o')
# plt.xlabel('Alpha (Weight for Collaborative)')
# plt.ylabel('Performance Metric')
# plt.title('Hybrid Performance vs Alpha')
# plt.grid(True, alpha=0.3)
# plt.show()

## Save Results

In [ ]:
# Save hybrid recommendations
# hybrid_recs.to_csv('../results/hybrid_recommendations.csv', index=False)
# print("Saved hybrid recommendations")